In [ ]:
import os
os.chdir('..')

from util import CONFIG
CONFIG.set_use_mpl_tables(True)

## Section 8.1

#### Section 8.1.1

In [ ]:
predicate = lambda k: True if k == 3 else False

In [ ]:
n = 3
print(f'\nGood outcomes: {[k for k in range(2**n) if predicate(k)]}')

In [ ]:
def c_phase_oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1

In [ ]:
from math import sqrt

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

In [ ]:
from util import print_state_table

print_state_table(state)

In [ ]:
c_phase_oracle(state, predicate)

In [ ]:
print('\nState after oracle is applied, changing the direction of good outcomes')
print_state_table(state)

#### Section 8.1.2

In [ ]:
def c_bit_oracle(state, predicate):
    N = len(state)
    state = state + [0 for _ in range(N)]
    for item in range(N):
        if predicate(item):
            state[N + item] = state[item]
            state[item] = 0
    return state

In [ ]:
predicate = lambda k: True if k == 3 else False

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

tag_state = c_bit_oracle(state, predicate)

In [ ]:
print_state_table(tag_state)

In [ ]:
from util import generate_state

n = 3
state = generate_state(n, seed=777)
print_state_table(state)

In [ ]:
state = c_bit_oracle(state, predicate)
print_state_table(state)

## Section 8.2

#### Section 8.2.1

In [ ]:
from math import pi
from sim_circuit import QuantumRegister, QuantumCircuit 

    
def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

In [ ]:
n = 3
items = [3]

oracle_circuit = phase_oracle_match(n, items)

In [ ]:
from util_qiskit import print_circuit

print_circuit(oracle_circuit)

In [ ]:
q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(n):
    qc.h(q[i])

In [ ]:
qc.append(oracle_circuit, QuantumRegister(n))

In [ ]:
print_state_table(qc.run())

In [ ]:
n = 3
items = [1, 3, 5]

oracle_circuit = phase_oracle_match(n, items)

In [ ]:
print_circuit(oracle_circuit)

In [ ]:
q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(n):
    qc.h(q[i])
    
qc.append(oracle_circuit, QuantumRegister(n))    

In [ ]:
n = 3

rs = generate_state(n, seed=777)
print_state_table(rs)

In [ ]:
q = QuantumRegister(n)
qc = QuantumCircuit(q)

qc.initialize(rs)
qc.append(oracle_circuit, QuantumRegister(n))

state = qc.run()
print_state_table(state)

#### Section 8.2.2

In [ ]:
def bit_oracle_match(n, items):
    q = QuantumRegister(n)
    a = QuantumRegister(1)
    qc = QuantumCircuit(q, a)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcx([q[i] for i in range(len(q))], a[0])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

In [ ]:
n = 3
items = [3]

oracle_circuit = bit_oracle_match(n, items)

# print_circuit(oracle_circuit)
from util_qiskit import draw_circuit

draw_circuit(oracle_circuit)

In [ ]:
q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

for i in range(n):
    qc.h(q[i])

qc.append(oracle_circuit, QuantumRegister(n+1))

In [ ]:
print_state_table(qc.run())

In [ ]:
n = 3
items = [1, 3, 5]

oracle_circuit = bit_oracle_match(n, items)

q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

for i in range(n):
    qc.h(q[i])

qc.append(oracle_circuit, QuantumRegister(n+1))

In [ ]:
print_state_table(qc.run())

## Section 8.3

In [ ]:
def phase_to_bit_oracle(oracle_circuit):
    n = sum(oracle_circuit.regs)
    q = QuantumRegister(n)
    a = QuantumRegister(1)
    qc = QuantumCircuit(q, a)
    qc.h(a[0])
    qc.c_append(oracle_circuit, a[0], q)
    qc.h(a[0])

    return qc

In [ ]:
n = 3
items = [1, 3, 5]
oracle_circuit = phase_oracle_match(n, items)

In [ ]:
print_circuit(oracle_circuit)

In [ ]:
state = generate_state(n, seed=777) + [0 for _ in range(2**n)]
print_state_table(state)

In [ ]:
q = QuantumRegister(n)
a = QuantumRegister(1)

qc = QuantumCircuit(q, a)
qc.initialize(state.copy())

qc.append(phase_to_bit_oracle(oracle_circuit), QuantumRegister(n+1))

In [ ]:
print_state_table(qc.run())

#### Section 8.3.2

In [ ]:
def bit_to_phase_oracle(oracle_circuit):
    n = sum(oracle_circuit.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    qc.append(oracle_circuit, q)
    qc.p(pi, q[len(q)-1])
    qc.append(oracle_circuit, q)

    return qc

In [ ]:
n = 3
items = [1, 3, 5]
oracle_circuit = bit_oracle_match(n, items)

In [ ]:
print_circuit(oracle_circuit)

In [ ]:
draw_circuit(oracle_circuit)

In [ ]:
n = 3
items = [1, 3, 5]
oracle_circuit = bit_oracle_match(n, items)

state = generate_state(n, seed=777) + [0 for _ in range(2**n)]

q = QuantumRegister(n)
a = QuantumRegister(1)

qc = QuantumCircuit(q, a)
qc.initialize(state.copy())

qc.append(bit_to_phase_oracle(oracle_circuit), QuantumRegister(n+1))

In [ ]:
print_state_table(qc.run())

## Section 8.4

In [ ]:
def recursive_fib(n):
    if n <= 1:
        return n
    else:
        return recursive_fib(n - 1) + recursive_fib(n - 2)

In [ ]:
[recursive_fib(n) for n in range(10)]

In [ ]:
from math import asin

def fib_circuit(n):
    theta = 2*asin((sqrt(5) - 1)/2)

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.ry(theta, q[i])

    for i in range(n - 1):
        qc.cry(-theta, q[i], q[i + 1])

    return qc

In [ ]:
qc = fib_circuit(1)
state = qc.run()

In [ ]:
print_state_table(state)

In [ ]:
qc = fib_circuit(2)
state = qc.run()

In [ ]:
print_state_table(state)

In [ ]:
from util import is_close

qc = fib_circuit(2)
state = qc.run()

assert is_close(abs(state[0])**2/abs(state[2])**2, (1+sqrt(5))/2)
assert is_close(abs(state[1])**2/abs(state[2])**2, (1+sqrt(5))/2)

In [ ]:
qc = fib_circuit(3)
state = qc.run()

In [ ]:
print_state_table(state)